In [ ]:
!pip install transformers accelerate pillow pandas scikit-learn tqdm

import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import collections

from google.colab import drive
drive.mount('/content/drive')
image_dir = "/content/drive/MyDrive/Dissertation/images"
csv_path = "/content/drive/MyDrive/Dissertation/labels.csv"

Mounted at /content/drive


In [ ]:
df = pd.read_csv(csv_path, encoding="ISO-8859-1")
print("dataset length:", len(df))

classes = ["very_positive", "positive", "neutral", "negative", "very_negative"]

keyword2class = {
    "very_positive": [
        "ecstatic", "overjoyed", "thrilled", "delighted", "elated",
        "extremely happy", "super happy", "euphoric", "radiant", "joyful",
        "laughing", "smiling broadly", "cheerful", "hilarious"
    ],
    "positive": [
        "happy", "smile", "content", "satisfied", "optimistic",
        "hopeful", "relaxed", "calm and happy", "pleasant",
        "friendly", "peaceful", "good mood"
    ],
    "neutral": [
        "neutral", "indifferent", "calm", "serious", "expressionless",
        "blank face", "no strong emotion", "objective", "ordinary"
    ],
    "negative": [
        "sad", "disappointed", "angry", "annoyed", "upset",
        "frowning", "irritated", "unhappy", "frustrated", "tired",
        "stressed", "bored", "worried", "lonely"
    ],
    "very_negative": [
        "crying", "depressed", "miserable", "heartbroken", "furious",
        "devastated", "terrified", "screaming", "hopeless", "grief",
        "extremely angry", "rage", "panic", "suffering"
    ]
}

model_id = "vikhyatk/moondream2"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
).eval()

def predict_sentiment(img_path):
    try:
        image = Image.open(img_path).convert("RGB")

        question = (
            "You are an expert in image sentiment classifier. "
            "Look at the image and classify the emotion strictly into one of these 5 categories: "
            "very_positive, positive, neutral, negative, very_negative. "
            "Answer with ONLY the category name, nothing else."
        )

        enc_image = model.encode_image(image)
        output = model.answer_question(enc_image, question, tokenizer)
        pred = output.lower().strip()

        # Mapping to legal categories
        for c in classes:
            if c in pred:
                return c
        return "neutral"
    except Exception as e:
        print(f"error {img_path}: {e}")
        return "neutral"

y_true, y_pred = [], []

for _, row in tqdm(df.iterrows(), total=len(df)):
    fname = row["image_name"]
    true_label = row["overall_sentiment"].strip().lower()
    img_path = os.path.join(image_dir, fname)

    if os.path.exists(img_path):
        pred_label = predict_sentiment(img_path)
    else:
        print(f"File not exists: {img_path}")
        pred_label = "neutral"

    y_true.append(true_label)
    y_pred.append(pred_label)

print("\nExamples of the first three images")
for i in range(min(3, len(y_true))):
    print(f"Image {i+1}: True label = {y_true[i]}, Prediction label = {y_pred[i]}")

acc = accuracy_score(y_true, y_pred)
f1_weighted = f1_score(y_true, y_pred, average="weighted")
f1_macro = f1_score(y_true, y_pred, average="macro")

print("Accuracy:", acc)
print("Weighted F1:", f1_weighted)
print("Macro F1:", f1_macro)

dataset length: 6992


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

 95%|█████████▌| 6676/6992 [1:17:16<04:34,  1.15it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 6992/6992 [1:20:42<00:00,  1.44it/s]


=== Examples of the first three images ===
Image 1: True label = very_positive, Prediction label = negative
Image 2: True label = very_positive, Prediction label = positive
Image 3: True label = positive, Prediction label = negative
Accuracy: 0.1875
Weighted F1: 0.17457238576024497
Macro F1: 0.10102940074402467
